# Using DuckDB in Python: A Comprehensive Guide

### Introduction to DuckDB

DuckDB is a high-performance, in-memory/in-process analytical database management system designed to execute complex analytical SQL queries fast, efficiently, and reliably over large datasets. It is often referred to as the "SQLite for analytics" due to its lightweight nature and ease of integration, making it ideal for analytics tasks, able to run entirely in memory or within an application.

It basically means that, DuckDB can process data fast, similar to traditional databases like PostgreSQL or SQLite, but without the need for an external server process. DuckDB is particularly well-suited for data analysis tasks, making it a powerful tool for data scientists and analysts.

### **Why DuckDB?**

* **In-Process DB** : DuckDB can be embedded directly into your Python environment, which means you don't need to manage a separate database server.
* **Columnar Storage** : It stores data in a columnar format, optimized for analytical queries.
* **SQL support** : DuckDB fully supports SQL queries, making it easy to interact with large datasets using well-known SQL syntax.
* **Fast and efficient** : DuckDB is designed for speed, particularly for analytical workloads like large aggregations or filtering operations.
* **Compatible with Pandas, Parquet, and Arrow** : It supports modern data formats, enabling seamless interaction with other data science libraries.

Let's explore how to use DuckDB in Python, going from installation to performing various operations like loading data, querying, and interacting with other Python libraries.

### Installation

To get started with DuckDB in Python, you need to install the DuckDB Python package. You can do this using `pip` or `conda`, depending on your environment:

```bash
pip install duckdb
```

or

```bash
conda install python-duckdb -c conda-forge
```

### Creating a DuckDB Database

In DuckDB, databases are either stored as files or kept in memory. For simplicity, let's first work with an **in-memory** database.

```python
import duckdb as dd

# Create an in-memory DuckDB connection
con = dd.connect(':memory:')
```

In [1]:
import duckdb as dd

# Create a persistent DuckDB database
con = dd.connect('my_database.db')

In [2]:
import duckdb as dd

# Running a basic SQL query
result = dd.sql("SELECT 'DuckDB_is_cool' AS answer").fetchall()
print(type(result))

<class 'list'>


In [3]:
import duckdb as dd

# Running a basic SQL query
result = dd.sql("SELECT 'DuckDB_is_cool' AS answer")
print( type(result) )
print(result)

<class 'duckdb.duckdb.DuckDBPyRelation'>
┌────────────────┐
│     answer     │
│    varchar     │
├────────────────┤
│ DuckDB_is_cool │
└────────────────┘



In [4]:
import duckdb as db

# Create a relation from a SQL query
rel = db.sql("SELECT * FROM range(10_00) AS tbl(ID)")

# Display the relation
rel.show()


┌────────────┐
│     ID     │
│   int64    │
├────────────┤
│          0 │
│          1 │
│          2 │
│          3 │
│          4 │
│          5 │
│          6 │
│          7 │
│          8 │
│          9 │
│          · │
│          · │
│          · │
│        990 │
│        991 │
│        992 │
│        993 │
│        994 │
│        995 │
│        996 │
│        997 │
│        998 │
│        999 │
├────────────┤
│ 1000 rows  │
│ (20 shown) │
└────────────┘



## Running SQL Queries & Data Ingestion

DuckDB supports standard SQL syntax, so you can run any SQL query with ease. Let's start by creating a on file DataBase and looking at it.

In [5]:
import duckdb as dd

# Create / connect to database
con = dd.connect('my_database.db')
con.sql('SHOW ALL TABLES')

┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

### Let's start by creating a table and inserting some data, manually.

#### Example 1: Creating a Table and Inserting Data Manually

In [6]:
# Create a table
con.execute('''
CREATE OR REPLACE TABLE countries (
    country VARCHAR,
    code VARCHAR,
    region VARCHAR,
    sub_region VARCHAR,
    intermediate_region VARCHAR
);
''')

# Insert some data
con.execute('''
INSERT INTO countries VALUES
('Australia', 'AUS', 'Oceania', 'Australia and New Zealand', ''),
('India', 'IND', 'Asia', 'Southern Asia', '');
''')

con.sql('SHOW ALL TABLES')

┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

#### Example 2: Creating a Table and Inserting Data Manually

In [7]:
# Create second table
con.execute('''
CREATE OR REPLACE TABLE employees (
    id INTEGER,
    name VARCHAR,
    age INTEGER,
    salary DOUBLE
);
''')

# Insert some data in second table
con.execute('''
INSERT INTO employees VALUES
(1, 'Person 1', 30, 70000),
(2, 'Person 2', 25, 55000),
(3, 'Person 3', 35, 80000);
''')


# Result of show tables after creating a table
con.sql('SHOW ALL TABLES')

┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

### Data Ingestion directly from files

DuckDB can ingest data from various formats, including CSV, Parquet, and JSON files. Let's start by removing the data data added manually to the countries table

In [8]:
con.sql('DELETE FROM countries;')
con.sql('SELECT * FROM countries;')

┌─────────┬─────────┬─────────┬────────────┬─────────────────────┐
│ country │  code   │ region  │ sub_region │ intermediate_region │
│ varchar │ varchar │ varchar │  varchar   │       varchar       │
├─────────┴─────────┴─────────┴────────────┴─────────────────────┤
│                             0 rows                             │
└────────────────────────────────────────────────────────────────┘

Now, let's insert all the values from the `countires.csv` file directly into the table

In [9]:
con.sql('''
        INSERT INTO countries (country, code, region, sub_region, intermediate_region) 
        (SELECT * FROM "countries.csv")
''')
con.sql('SELECT * FROM countries LIMIT 5;')

┌────────────────┬─────────┬─────────┬─────────────────┬─────────────────────┐
│    country     │  code   │ region  │   sub_region    │ intermediate_region │
│    varchar     │ varchar │ varchar │     varchar     │       varchar       │
├────────────────┼─────────┼─────────┼─────────────────┼─────────────────────┤
│ Afghanistan    │ AFG     │ Asia    │ Southern Asia   │ NULL                │
│ Åland Islands  │ ALA     │ Europe  │ Northern Europe │ NULL                │
│ Albania        │ ALB     │ Europe  │ Southern Europe │ NULL                │
│ Algeria        │ DZA     │ Africa  │ Northern Africa │ NULL                │
│ American Samoa │ ASM     │ Oceania │ Polynesia       │ NULL                │
└────────────────┴─────────┴─────────┴─────────────────┴─────────────────────┘

### Working with persistent data stored in files

In [10]:
con.sql('''
        SELECT  *
            FROM
                countries
            WHERE
                region = 'Oceania'
                AND sub_region = 'Australia and New Zealand'
''')

┌───────────────────────────────────┬─────────┬─────────┬───────────────────────────┬─────────────────────┐
│              country              │  code   │ region  │        sub_region         │ intermediate_region │
│              varchar              │ varchar │ varchar │          varchar          │       varchar       │
├───────────────────────────────────┼─────────┼─────────┼───────────────────────────┼─────────────────────┤
│ Australia                         │ AUS     │ Oceania │ Australia and New Zealand │ NULL                │
│ Christmas Island                  │ CXR     │ Oceania │ Australia and New Zealand │ NULL                │
│ Cocos (Keeling) Islands           │ CCK     │ Oceania │ Australia and New Zealand │ NULL                │
│ Heard Island and McDonald Islands │ HMD     │ Oceania │ Australia and New Zealand │ NULL                │
│ New Zealand                       │ NZL     │ Oceania │ Australia and New Zealand │ NULL                │
│ Norfolk Island            

### Working with Data directly from files

While you can DuckDB can ingest data from various formats, as discussed above. DuckDB also gives an provision to read these files into DuckDB relations (tables) and query them directly:

In [11]:
relation = con.sql('''
        SELECT  *
            FROM
                'countries.csv'
            WHERE
                region = 'Oceania'
                AND "sub-region" = 'Polynesia'
''')
relation

┌───────────────────┬─────────┬─────────┬────────────┬─────────────────────┐
│       name        │  code   │ region  │ sub-region │ intermediate-region │
│      varchar      │ varchar │ varchar │  varchar   │       varchar       │
├───────────────────┼─────────┼─────────┼────────────┼─────────────────────┤
│ American Samoa    │ ASM     │ Oceania │ Polynesia  │ NULL                │
│ Cook Islands      │ COK     │ Oceania │ Polynesia  │ NULL                │
│ French Polynesia  │ PYF     │ Oceania │ Polynesia  │ NULL                │
│ Niue              │ NIU     │ Oceania │ Polynesia  │ NULL                │
│ Pitcairn          │ PCN     │ Oceania │ Polynesia  │ NULL                │
│ Samoa             │ WSM     │ Oceania │ Polynesia  │ NULL                │
│ Tokelau           │ TKL     │ Oceania │ Polynesia  │ NULL                │
│ Tonga             │ TON     │ Oceania │ Polynesia  │ NULL                │
│ Tuvalu            │ TUV     │ Oceania │ Polynesia  │ NULL                │

These connections working with data from files directly are known as DuckDB relation objects.

In [12]:
print(type(relation))

<class 'duckdb.duckdb.DuckDBPyRelation'>


We can display all data in these `relations`, as demonstrated above of extract them as a list of tuples using `fetchall` method of these relations.

In [13]:
relation.fetchall()

[('American Samoa', 'ASM', 'Oceania', 'Polynesia', None),
 ('Cook Islands', 'COK', 'Oceania', 'Polynesia', None),
 ('French Polynesia', 'PYF', 'Oceania', 'Polynesia', None),
 ('Niue', 'NIU', 'Oceania', 'Polynesia', None),
 ('Pitcairn', 'PCN', 'Oceania', 'Polynesia', None),
 ('Samoa', 'WSM', 'Oceania', 'Polynesia', None),
 ('Tokelau', 'TKL', 'Oceania', 'Polynesia', None),
 ('Tonga', 'TON', 'Oceania', 'Polynesia', None),
 ('Tuvalu', 'TUV', 'Oceania', 'Polynesia', None),
 ('Wallis and Futuna', 'WLF', 'Oceania', 'Polynesia', None)]

## Integrating with Pandas

One of DuckDB’s most powerful features is its compatibility with `Pandas`. You can run SQL queries directly on Pandas DataFrames or convert query results into DataFrames.

Example: Converting to Pandas DataFrames

In [14]:
relation.to_df()

,name,code,region,sub-region,intermediate-region
0,American Samoa,ASM,Oceania,Polynesia,None
1,Cook Islands,COK,Oceania,Polynesia,None
2,French Polynesia,PYF,Oceania,Polynesia,None
3,Niue,NIU,Oceania,Polynesia,None
4,Pitcairn,PCN,Oceania,Polynesia,None
5,Samoa,WSM,Oceania,Polynesia,None
6,Tokelau,TKL,Oceania,Polynesia,None
7,Tonga,TON,Oceania,Polynesia,None
8,Tuvalu,TUV,Oceania,Polynesia,None
9,Wallis and Futuna,WLF,Oceania,Polynesia,None


Example: Querying Pandas DataFrames directly

In [16]:
import pandas as pd

df = pd.DataFrame({
    'id': [4, 5, 6],
    'name': ['Person 4', 'Person 5', 'Person 6'],
    'age': [45, 40, 35],
    'salary': [100000, 85000, 75000]
})

con.sql('''
        INSERT INTO employees (id, name, age, salary)
        SELECT * FROM df
''')

con.sql('select * from employees')

┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     1 │ Person 1 │    30 │  70000.0 │
│     2 │ Person 2 │    25 │  55000.0 │
│     3 │ Person 3 │    35 │  80000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     5 │ Person 5 │    40 │  85000.0 │
│     6 │ Person 6 │    35 │  75000.0 │
└───────┴──────────┴───────┴──────────┘

DuckDB allows you to run SQL queries directly on a Pandas DataFrame. You can convert query results back into DataFrames using .df().

In [18]:
con.sql('SELECT * FROM df').df()

,id,name,age,salary
0,4,Person 4,45,100000
1,5,Person 5,40,85000
2,6,Person 6,35,75000


## Working with Parquet and Arrow

DuckDB also supports efficient handling of Parquet and Arrow formats, commonly used in big data scenarios. You can read data from Parquet files and run SQL queries on them without first loading them into memory.

Example: Reading from Parquet Files

In [19]:
# Read data from a Parquet file
con.sql("SELECT * FROM 'countries.parquet'").df().head()

,name,code,region,sub-region,intermediate-region
0,Afghanistan,AFG,Asia,Southern Asia,None
1,Åland Islands,ALA,Europe,Northern Europe,None
2,Albania,ALB,Europe,Southern Europe,None
3,Algeria,DZA,Africa,Northern Africa,None
4,American Samoa,ASM,Oceania,Polynesia,None


Similarly, DuckDB integrates well with Apache Arrow and supports operations on Arrow tables in a similar way.

### Advanced Queries and Using DuckDB for Analytical Workloads

DuckDB is optimized for performance, especially for analytical queries. DuckDB's architecture, particularly its use of vectorized execution and columnar storage, helps DuckDB to speed up query processing and make it extremely efficient for data analytics. Additionally, DuckDB can operate directly on compressed data formats like Parquet, reducing the need for data decompression.

- Window Functions: You can perform windowing operations (e.g., running totals, moving averages).
- Group By: Complex group-by operations with large datasets are optimized.
- Parallel Execution: DuckDB automatically parallelizes many operations for faster results on large datasets.


Example: Group and count countries by their regions.

In [20]:
con.sql('''
        SELECT  region
                , COUNT(DISTINCT country) AS country_counts
            FROM
                countries
            GROUP BY
                region
            ORDER BY
                country_counts DESC
''')

┌──────────┬────────────────┐
│  region  │ country_counts │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ Africa   │             60 │
│ Americas │             57 │
│ Europe   │             51 │
│ Asia     │             51 │
│ Oceania  │             29 │
│ NULL     │              1 │
└──────────┴────────────────┘

Example: Calculate the average salary and find people with above avg salary

In [21]:

con.sql('''
        WITH avg_salary AS (
            SELECT
                    ROUND(AVG(salary),2) AS avg_salary
                FROM
                    employees
            )
        
        SELECT
                *
            FROM
                employees
            WHERE
                salary > (SELECT avg_salary FROM avg_salary)
        
''')


┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     3 │ Person 3 │    35 │  80000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     5 │ Person 5 │    40 │  85000.0 │
└───────┴──────────┴───────┴──────────┘

## Speed Comparision: DuckDB vs Pandas

DuckDB and Pandas are both popular tools for data manipulation in Python, but they have different strengths. Pandas is a general-purpose data manipulation library optimized for in-memory operations, whereas DuckDB is designed specifically for high-performance, analytical queries on large datasets using SQL. DuckDB's columnar storage and query optimization techniques make it significantly faster than Pandas for complex and large-scale analytical queries.

### Why is DuckDB Faster for Analytical Queries?
- *Columnar Storage:* DuckDB stores data in a columnar format, which is more efficient for analytical queries (like filtering and aggregations). Pandas stores data in row-major format, which is better for general-purpose operations but can be slower for these specific tasks.

- *Query Optimization:* DuckDB uses query optimizations like predicate pushdown, efficient joins, and parallel query execution, which are typically missing or less efficient in Pandas.

- *Parallel Execution:* DuckDB can automatically parallelize complex queries, leveraging multiple cores in modern CPUs, while Pandas processes data mostly single-threaded by default.

- *On-Disk Storage:* DuckDB efficiently handles datasets that don’t fit into memory by using on-disk storage formats like Parquet, while Pandas requires that all data fits into memory.

In [22]:
import duckdb as dd
import pandas as pd
import numpy as np
import time

# Generate a large random dataset
data_size = 10**8  # 10 million rows
df = pd.DataFrame({
    'id': np.arange(data_size),
    'value': np.random.randn(data_size)
})

# Scenario: Filter rows where value is greater than 1

# Filtering using Pandas
start = time.time()
pandas_filtered = df[df['value'] > 1]
end = time.time()
pandas_time = end - start
print(f"Pandas filtering time: {pandas_time:.2f} seconds")

# Filtering using DuckDB
start = time.time()
duckdb_filtered = dd.query("SELECT * FROM df WHERE value > 1").df()
end = time.time()
duckdb_time = end - start
print(f"DuckDB filtering time: {duckdb_time:.2f} seconds")

print(f"DuckDB is {pandas_time / duckdb_time:.2f} times faster than Pandas")


Pandas filtering time: 0.93 seconds
DuckDB filtering time: 0.60 seconds
DuckDB is 1.55 times faster than Pandas


In [23]:
# Scenario: Calculate mean of 'value' column

# Aggregating using Pandas
start = time.time()
pandas_mean = df['value'].mean()
end = time.time()
pandas_time = end - start
print(f"Pandas filtering time: {pandas_time:.2f} seconds")

# Aggregating using DuckDB
start = time.time()
duckdb_mean = dd.query("SELECT AVG(value) FROM df").fetchone()[0]
end = time.time()
duckdb_time = end - start
print(f"DuckDB aggregation time: {duckdb_time:.2f} seconds")


print(f"DuckDB is {pandas_time / duckdb_time:.2f} times faster than Pandas")

Pandas filtering time: 0.54 seconds
DuckDB aggregation time: 0.10 seconds
DuckDB is 5.27 times faster than Pandas


## Closing the Connection

Once you're done with your queries, always remember to close the DuckDB connection.

In [24]:
con.close()

## Conclusion

DuckDB is a powerful tool for performing efficient SQL operations in Python, especially when working with large datasets or complex analytical queries. Its ease of integration with modern data formats like Pandas, Parquet, and Arrow, combined with its fast performance, makes it a valuable addition to any data analyst's toolkit.

Whether you're building an in-memory database for fast analytics, working with small datasets or working with large-scale data in Parquet files, DuckDB can simplify the process and accelerate performance. Its SQL syntax is easy to learn, and its compatibility with Python makes it highly flexible for a wide range of data-related tasks.

Read more at [DuckDB Python API](https://duckdb.org/docs/api/python/overview):